In [ ]:
import json
from os import environ, makedirs, walk
from pathlib import Path, PurePath

from openai import AzureOpenAI
from openai.types.chat import (
    ChatCompletion,
)

In [ ]:
endpoint = environ["AZURE_OPENAI_ENDPOINT"]
api_key = environ["AZURE_OPENAI_KEY"]
api_version = "2023-09-01-preview"
model_deployment = "gpt-35-turbo-0613"

In [ ]:
data_dir = "data/"

In [ ]:
client = AzureOpenAI(
    azure_endpoint=endpoint, api_version=api_version, api_key=api_key
)

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are an expert Python developer "
        "with great experience in writing clear and efficient solutions. "
        "Use the provided tools to manage the files "
        "that you need to read or write.",
    },
    {
        "role": "user",
        "content": "Create a small Python package called `clock` "
        "that provides two classes.\n\n"
        "The first class is a `Clock` that implements the observable pattern. "
        "Each second, the `Clock` notifies all of its listeners "
        "that a `tick` happened.\n\n"
        "The second class of the package is `ClockPrinter`. "
        "It is meant to be a `Clock` listener. "
        "It provides a method `on_tick` that gets called by `Clock` "
        "each second and prints the time at which the tick happened.\n\n"
        "Make sure to write an example code file outside of the package that "
        "demonstrates how to use it.\n\n"
        "Start by writing a short numbered todo list of "
        "the tasks you need to do. "
        "Please call the `done` tool once you are done. "
        "Thanks!",
    },
]

In [ ]:
tools = [
    # List files
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns the list of all existing files.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    # Write file
    {
        "type": "function",
        "function": {
            "name": "write_file",
            "description": "Writes the provided text content to the "
            "provided file path. Returns 'OK' on success.",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "Path of the file to write to. "
                        "e.g. 'some_dir/my_file.py'",
                    },
                    "content": {
                        "type": "string",
                        "description": "The text content to write to the file.",
                    },
                },
                "required": ["file_path", "content"],
            },
        },
    },
    # Done
    {
        "type": "function",
        "function": {
            "name": "done",
            "description": "Call this once you are done with all your tasks.",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
]

In [ ]:
def list_files() -> str:
    result: list[str] = []
    for path, subdirs, files in walk(data_dir):
        for filename in files:
            result.append(str(PurePath(path, filename))[5:])
    return "\n".join(result)

In [ ]:
def write_file(file_path: str, content: str) -> str:
    path = Path(PurePath(data_dir, file_path))
    if not path.parent.exists():
        makedirs(path.parent.resolve())
    with open(path, "w", encoding="utf-8") as f:
        f.write(content)
    return "OK"

In [ ]:
done: list[bool] = [False]


def terminate() -> None:
    done[0] = True

In [ ]:
available_functions = {
    "list_files": list_files,
    "write_file": write_file,
    "done": terminate,
}

In [ ]:
def request_chat_completion(messages: list) -> ChatCompletion:
    return client.chat.completions.create(
        model=model_deployment,
        temperature=0.2,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

In [ ]:
def process_completion(completion: ChatCompletion) -> None:
    # Get response message
    response_message = completion.choices[0].message
    response_message.content = response_message.content or ""
    messages.append(response_message)

    # Call tools
    tool_calls = response_message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print(f"Calling {function_name}({function_args})...")
            function_response = function_to_call(**function_args)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response or "",
                }
            )

In [ ]:
while not done[0]:
    completion = request_chat_completion(messages)
    process_completion(completion)

In [ ]:
# Print messages
for message in messages:
    print(f"{type(message)}:\n{message}\n\n")

In [ ]:
messages.append({"role": "user", "content": "Thanks"})
done[0] = False